import各种类

In [3]:
import random
import os
import argparse
import math

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import nltk
nltk.download("all-nltk")
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from collections import Counter

from lib.DataLoader import *
from lib.criterion import *
from lib.loss import *
from lib.optimizer import *
from lib.parser import *
from lib.utils import *
from lib.Mytransformer import *
from evaluate import *

[nltk_data] Downloading collection 'all-nltk'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_da

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_

[nltk_data]    |   Package word2vec_sample is already up-to-date!
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package panlex_swadesh is already up-to-date!
[nltk_data]    | Downloading package mte_teip5 to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\ProgramData\Anaconda3\Lib\site-
[nltk_data]    |     packages\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[n

In [4]:
import copy
import warnings
warnings.filterwarnings("ignore")

模型构建

In [6]:
def make_model(src_vocab, lab_vocab, N = 6, d_model = 512, d_ff = 2048, h = 8, dropout = 0.1):
    c = copy.deepcopy
    multihead = MultiHeadedAttention(h, d_model).to(args.device)
    feedforward = FeedForward(d_model, d_ff, dropout).to(args.device)
    position = PositionalEncoding(d_model, dropout).to(args.device)
    model = Transformer(
        Encoder(EncoderLayer(d_model, c(multihead), c(feedforward), dropout).to(args.device), N).to(args.device),
        Decoder(DecoderLayer(d_model, c(multihead), c(multihead), c(feedforward), dropout).to(args.device), N).to(args.device),
        nn.Sequential(Embeddings(d_model, src_vocab).to(args.device), c(position)),
        nn.Sequential(Embeddings(d_model, lab_vocab).to(args.device), c(position)),
        Generator(d_model, lab_vocab)).to(args.device)
  
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model.to(args.device)

参数设置

In [8]:
args.layers = 2
args.batch_size = 128
args.d_model = 256
args.d_ff = 1024
args.h_num = 8
args.dropout = 0.1
set_seed(10)

加载数据

In [11]:
Data = DataHandler()

模型创建

In [9]:
args.src_vocab = 131074 
args.lab_vocab = 7582 
model = make_model(args.src_vocab, args.lab_vocab, args.layers, args.d_model, args.d_ff, args.h_num, args.dropout)
# print(model)

测试模型

In [15]:
g_max_seq3 = float("inf")
Data.prepare_test(g_max_seq3)
model.load_state_dict(torch.load("model.pkl"))
evaluate(Data, model)

100000
200000
300000
400000
100000
200000
300000
400000
-------Test Data Loaded!------
----------Identify!-----------
测试样本: 100,	 bleu1: 0.3077	 bleu2: 0.1601	 bleu3: 0.0615	 bleu4: 0.0391
	 100 sentences has been translated!
测试样本: 200,	 bleu1: 0.5	 bleu2: 0.2449	 bleu3: 0.063	 bleu4: 0.0323
	 200 sentences has been translated!
测试样本: 300,	 bleu1: 0.6087	 bleu2: 0.4074	 bleu3: 0.2873	 bleu4: 0.1856
	 300 sentences has been translated!
测试样本: 400,	 bleu1: 0.4222	 bleu2: 0.2771	 bleu3: 0.2074	 bleu4: 0.1436
	 400 sentences has been translated!
测试样本: 500,	 bleu1: 0.3125	 bleu2: 0.1443	 bleu3: 0.053	 bleu4: 0.0327
	 500 sentences has been translated!
测试样本: 600,	 bleu1: 0.5023	 bleu2: 0.296	 bleu3: 0.1385	 bleu4: 0.0539
	 600 sentences has been translated!
测试样本: 700,	 bleu1: 0.6296	 bleu2: 0.4921	 bleu3: 0.4078	 bleu4: 0.326
	 700 sentences has been translated!
测试样本: 800,	 bleu1: 0.5903	 bleu2: 0.1678	 bleu3: 0.0521	 bleu4: 0.0294
	 800 sentences has been translated!
测试样本: 900,	 bleu1: 0.5406

测试样本: 7500,	 bleu1: 0.4994	 bleu2: 0.3833	 bleu3: 0.3223	 bleu4: 0.2798
	 7500 sentences has been translated!
测试样本: 7600,	 bleu1: 0.6286	 bleu2: 0.471	 bleu3: 0.361	 bleu4: 0.2769
	 7600 sentences has been translated!
测试样本: 7700,	 bleu1: 0.4	 bleu2: 0.3162	 bleu3: 0.2354	 bleu4: 0.156
	 7700 sentences has been translated!
测试样本: 7800,	 bleu1: 0.5471	 bleu2: 0.4479	 bleu3: 0.3648	 bleu4: 0.289
	 7800 sentences has been translated!
测试样本: 7900,	 bleu1: 0.375	 bleu2: 0.3128	 bleu3: 0.261	 bleu4: 0.2245
	 7900 sentences has been translated!
测试样本: 8000,	 bleu1: 0.4333	 bleu2: 0.3457	 bleu3: 0.2575	 bleu4: 0.0892
	 8000 sentences has been translated!
测试样本: 8100,	 bleu1: 0.6	 bleu2: 0.3536	 bleu3: 0.1758	 bleu4: 0.0705
	 8100 sentences has been translated!
测试样本: 8200,	 bleu1: 0.3846	 bleu2: 0.2464	 bleu3: 0.1701	 bleu4: 0.1081
	 8200 sentences has been translated!
测试样本: 8300,	 bleu1: 0.5	 bleu2: 0.4	 bleu3: 0.2988	 bleu4: 0.2194
	 8300 sentences has been translated!
测试样本: 8400,	 bleu1: 0.549	 b

测试样本: 15000,	 bleu1: 0.6458	 bleu2: 0.5622	 bleu3: 0.479	 bleu4: 0.4049
	 15000 sentences has been translated!
测试样本: 15100,	 bleu1: 0.4231	 bleu2: 0.3187	 bleu3: 0.1617	 bleu4: 0.0655
	 15100 sentences has been translated!
测试样本: 15200,	 bleu1: 0.4993	 bleu2: 0.3673	 bleu3: 0.2658	 bleu4: 0.1523
	 15200 sentences has been translated!
测试样本: 15300,	 bleu1: 0.5	 bleu2: 0.2294	 bleu3: 0.0664	 bleu4: 0.0362
	 15300 sentences has been translated!
测试样本: 15400,	 bleu1: 0.375	 bleu2: 0.2236	 bleu3: 0.1529	 bleu4: 0.0724
	 15400 sentences has been translated!
测试样本: 15500,	 bleu1: 0.5322	 bleu2: 0.4435	 bleu3: 0.357	 bleu4: 0.2461
	 15500 sentences has been translated!
测试样本: 15600,	 bleu1: 0.5789	 bleu2: 0.401	 bleu3: 0.2115	 bleu4: 0.0877
	 15600 sentences has been translated!
测试样本: 15700,	 bleu1: 0.1	 bleu2: 0.0229	 bleu3: 0.0143	 bleu4: 0.0115
	 15700 sentences has been translated!
测试样本: 15800,	 bleu1: 0.6522	 bleu2: 0.5247	 bleu3: 0.397	 bleu4: 0.3177
	 15800 sentences has been translated!
测试样

测试样本: 22400,	 bleu1: 0.8108	 bleu2: 0.7233	 bleu3: 0.6539	 bleu4: 0.5957
	 22400 sentences has been translated!
测试样本: 22500,	 bleu1: 0.3448	 bleu2: 0.2936	 bleu3: 0.2518	 bleu4: 0.2226
	 22500 sentences has been translated!
测试样本: 22600,	 bleu1: 0.8105	 bleu2: 0.7504	 bleu3: 0.6968	 bleu4: 0.6597
	 22600 sentences has been translated!
测试样本: 22700,	 bleu1: 0.4848	 bleu2: 0.3482	 bleu3: 0.2272	 bleu4: 0.0791
	 22700 sentences has been translated!
测试样本: 22800,	 bleu1: 0.5556	 bleu2: 0.4134	 bleu3: 0.2737	 bleu4: 0.171
	 22800 sentences has been translated!
测试样本: 22900,	 bleu1: 0.04	 bleu2: 0.0129	 bleu3: 0.009	 bleu4: 0.0076
	 22900 sentences has been translated!
测试样本: 23000,	 bleu1: 0.5769	 bleu2: 0.4557	 bleu3: 0.3731	 bleu4: 0.3259
	 23000 sentences has been translated!
测试样本: 23100,	 bleu1: 0.4334	 bleu2: 0.2404	 bleu3: 0.161	 bleu4: 0.0757
	 23100 sentences has been translated!
测试样本: 23200,	 bleu1: 0.2353	 bleu2: 0.1213	 bleu3: 0.0461	 bleu4: 0.0289
	 23200 sentences has been translate

测试样本: 29800,	 bleu1: 0.4704	 bleu2: 0.4013	 bleu3: 0.3361	 bleu4: 0.2757
	 29800 sentences has been translated!
测试样本: 29900,	 bleu1: 0.6882	 bleu2: 0.5731	 bleu3: 0.4748	 bleu4: 0.4184
	 29900 sentences has been translated!
测试样本: 30000,	 bleu1: 0.3478	 bleu2: 0.2812	 bleu3: 0.196	 bleu4: 0.0783
	 30000 sentences has been translated!
测试样本: 30100,	 bleu1: 0.3149	 bleu2: 0.1141	 bleu3: 0.0384	 bleu4: 0.0226
	 30100 sentences has been translated!
测试样本: 30200,	 bleu1: 0.69	 bleu2: 0.5553	 bleu3: 0.4057	 bleu4: 0.2734
	 30200 sentences has been translated!
测试样本: 30300,	 bleu1: 0.52	 bleu2: 0.3606	 bleu3: 0.2569	 bleu4: 0.1666
	 30300 sentences has been translated!
测试样本: 30400,	 bleu1: 0.4783	 bleu2: 0.2949	 bleu3: 0.2023	 bleu4: 0.0802
	 30400 sentences has been translated!
测试样本: 30500,	 bleu1: 0.5833	 bleu2: 0.4606	 bleu3: 0.2768	 bleu4: 0.1239
	 30500 sentences has been translated!
测试样本: 30600,	 bleu1: 0.4762	 bleu2: 0.3233	 bleu3: 0.1987	 bleu4: 0.1191
	 30600 sentences has been translate

...

测试样本: 366600,	 bleu1: 0.5294	 bleu2: 0.2572	 bleu3: 0.0761	 bleu4: 0.0421
	 366600 sentences has been translated!
测试样本: 366700,	 bleu1: 0.4167	 bleu2: 0.301	 bleu3: 0.1603	 bleu4: 0.0665
	 366700 sentences has been translated!
测试样本: 366800,	 bleu1: 0.3913	 bleu2: 0.2667	 bleu3: 0.1892	 bleu4: 0.0763
	 366800 sentences has been translated!
测试样本: 366900,	 bleu1: 0.625	 bleu2: 0.571	 bleu3: 0.511	 bleu4: 0.4418
	 366900 sentences has been translated!
测试样本: 367000,	 bleu1: 0.1515	 bleu2: 0.0688	 bleu3: 0.0248	 bleu4: 0.015
	 367000 sentences has been translated!
测试样本: 367100,	 bleu1: 0.4304	 bleu2: 0.3122	 bleu3: 0.167	 bleu4: 0.0697
	 367100 sentences has been translated!
测试样本: 367200,	 bleu1: 0.3235	 bleu2: 0.198	 bleu3: 0.1348	 bleu4: 0.0943
	 367200 sentences has been translated!
测试样本: 367300,	 bleu1: 0.4762	 bleu2: 0.3086	 bleu3: 0.0794	 bleu4: 0.0409
	 367300 sentences has been translated!
测试样本: 367400,	 bleu1: 0.5185	 bleu2: 0.3994	 bleu3: 0.2944	 bleu4: 0.1016
	 367400 sentences ha

测试样本: 373900,	 bleu1: 0.3571	 bleu2: 0.0524	 bleu3: 0.0284	 bleu4: 0.0214
	 373900 sentences has been translated!
测试样本: 374000,	 bleu1: 0.4444	 bleu2: 0.2357	 bleu3: 0.0926	 bleu4: 0.0603
	 374000 sentences has been translated!
测试样本: 374100,	 bleu1: 0.4211	 bleu2: 0.3746	 bleu3: 0.3208	 bleu4: 0.2535
	 374100 sentences has been translated!
测试样本: 374200,	 bleu1: 0.52	 bleu2: 0.4163	 bleu3: 0.3112	 bleu4: 0.1924
	 374200 sentences has been translated!
测试样本: 374300,	 bleu1: 0.1739	 bleu2: 0.0281	 bleu3: 0.0156	 bleu4: 0.0117
	 374300 sentences has been translated!
测试样本: 374400,	 bleu1: 0.5161	 bleu2: 0.3935	 bleu3: 0.2774	 bleu4: 0.1976
	 374400 sentences has been translated!
测试样本: 374500,	 bleu1: 0.25	 bleu2: 0.2085	 bleu3: 0.181	 bleu4: 0.1542
	 374500 sentences has been translated!
测试样本: 374600,	 bleu1: 0.57	 bleu2: 0.4347	 bleu3: 0.3226	 bleu4: 0.2558
	 374600 sentences has been translated!
测试样本: 374700,	 bleu1: 0.3742	 bleu2: 0.1581	 bleu3: 0.0565	 bleu4: 0.0344
	 374700 sentences ha

测试样本: 381200,	 bleu1: 0.5294	 bleu2: 0.4067	 bleu3: 0.321	 bleu4: 0.2622
	 381200 sentences has been translated!
测试样本: 381300,	 bleu1: 0.7333	 bleu2: 0.6055	 bleu3: 0.4832	 bleu4: 0.3703
	 381300 sentences has been translated!
测试样本: 381400,	 bleu1: 0.2308	 bleu2: 0.0439	 bleu3: 0.026	 bleu4: 0.0204
	 381400 sentences has been translated!
测试样本: 381500,	 bleu1: 0.3323	 bleu2: 0.1958	 bleu3: 0.1155	 bleu4: 0.0504
	 381500 sentences has been translated!
测试样本: 381600,	 bleu1: 0.3529	 bleu2: 0.1485	 bleu3: 0.0528	 bleu4: 0.032
	 381600 sentences has been translated!
测试样本: 381700,	 bleu1: 0.5871	 bleu2: 0.4288	 bleu3: 0.3333	 bleu4: 0.2274
	 381700 sentences has been translated!
测试样本: 381800,	 bleu1: 0.2018	 bleu2: 0.1318	 bleu3: 0.0929	 bleu4: 0.0447
	 381800 sentences has been translated!
测试样本: 381900,	 bleu1: 0.4737	 bleu2: 0.408	 bleu3: 0.3332	 bleu4: 0.2373
	 381900 sentences has been translated!
测试样本: 382000,	 bleu1: 0.1851	 bleu2: 0.1194	 bleu3: 0.083	 bleu4: 0.0393
	 382000 sentences 

测试样本: 388500,	 bleu1: 0.2763	 bleu2: 0.0942	 bleu3: 0.031	 bleu4: 0.018
	 388500 sentences has been translated!
测试样本: 388600,	 bleu1: 0.3125	 bleu2: 0.284	 bleu3: 0.2527	 bleu4: 0.2172
	 388600 sentences has been translated!
测试样本: 388700,	 bleu1: 0.3077	 bleu2: 0.2481	 bleu3: 0.2173	 bleu4: 0.1912
	 388700 sentences has been translated!
测试样本: 388800,	 bleu1: 0.3333	 bleu2: 0.0488	 bleu3: 0.0264	 bleu4: 0.0198
	 388800 sentences has been translated!
测试样本: 388900,	 bleu1: 0.0404	 bleu2: 0.0094	 bleu3: 0.006	 bleu4: 0.0049
	 388900 sentences has been translated!
测试样本: 389000,	 bleu1: 0.2222	 bleu2: 0.1594	 bleu3: 0.1143	 bleu4: 0.082
	 389000 sentences has been translated!
测试样本: 389100,	 bleu1: 0.2727	 bleu2: 0.114	 bleu3: 0.0402	 bleu4: 0.0242
	 389100 sentences has been translated!
测试样本: 389200,	 bleu1: 0.3636	 bleu2: 0.2279	 bleu3: 0.1375	 bleu4: 0.0608
	 389200 sentences has been translated!
测试样本: 389300,	 bleu1: 0.2799	 bleu2: 0.1843	 bleu3: 0.1197	 bleu4: 0.082
	 389300 sentences ha

测试样本: 395800,	 bleu1: 0.25	 bleu2: 0.0408	 bleu3: 0.0228	 bleu4: 0.0174
	 395800 sentences has been translated!
测试样本: 395900,	 bleu1: 0.3429	 bleu2: 0.2657	 bleu3: 0.1858	 bleu4: 0.0669
	 395900 sentences has been translated!
测试样本: 396000,	 bleu1: 0.2336	 bleu2: 0.1957	 bleu3: 0.1491	 bleu4: 0.111
	 396000 sentences has been translated!
测试样本: 396100,	 bleu1: 0.381	 bleu2: 0.276	 bleu3: 0.2291	 bleu4: 0.1912
	 396100 sentences has been translated!
测试样本: 396200,	 bleu1: 0.3333	 bleu2: 0.2182	 bleu3: 0.0716	 bleu4: 0.0418
	 396200 sentences has been translated!
测试样本: 396300,	 bleu1: 0.6277	 bleu2: 0.4914	 bleu3: 0.3646	 bleu4: 0.2525
	 396300 sentences has been translated!
测试样本: 396400,	 bleu1: 0.4868	 bleu2: 0.3894	 bleu3: 0.2936	 bleu4: 0.1973
	 396400 sentences has been translated!
测试样本: 396500,	 bleu1: 0.4412	 bleu2: 0.327	 bleu3: 0.2156	 bleu4: 0.0754
	 396500 sentences has been translated!
测试样本: 396600,	 bleu1: 0.5556	 bleu2: 0.4042	 bleu3: 0.3129	 bleu4: 0.2528
	 396600 sentences h